In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import tensorflow as tf
import numpy as np
from tensor2tensor.data_generators import problem_hparams
from tensor2tensor.models import evolved_transformer
from tensor2tensor.models import transformer
from tensor2tensor.utils import optimize

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [3]:
import json

with open('train-test-bpe.json') as fopen:
    dataset = json.load(fopen)

In [4]:
train_X = dataset['train_X']
train_Y = dataset['train_Y']
test_X = dataset['test_X']
test_Y = dataset['test_Y']

In [5]:
PAD = 0
EOS = 1
UNK = 2
GO = 3

In [6]:
class Translator:
    def __init__(self, size_layer, num_layers, embedded_size,
                 from_dict_size, to_dict_size, learning_rate, beam_width = 10):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        def attention(encoder_out, seq_len, reuse=False):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size_layer, 
                                                                    memory = encoder_out,
                                                                    memory_sequence_length = seq_len)
            return tf.contrib.seq2seq.AttentionWrapper(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells(reuse) for _ in range(num_layers)]), 
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        encoder_embedding = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embedding = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        
        encoder_out, encoder_state = tf.nn.dynamic_rnn(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
            inputs = tf.nn.embedding_lookup(encoder_embedding, self.X),
            sequence_length = self.X_seq_len,
            dtype = tf.float32)
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        dense = tf.layers.Dense(to_dict_size)
        
        with tf.variable_scope('decode'):
            decoder_cells = attention(encoder_out, self.X_seq_len)
            training_helper = tf.contrib.seq2seq.TrainingHelper(
                inputs = tf.nn.embedding_lookup(decoder_embedding, decoder_input),
                sequence_length = self.Y_seq_len,
                time_major = False)
            training_decoder = tf.contrib.seq2seq.BasicDecoder(
                    cell = decoder_cells,
                    helper = training_helper,
                    initial_state = decoder_cells.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state),
                    output_layer = dense)
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder = training_decoder,
                    impute_finished = True,
                    maximum_iterations = tf.reduce_max(self.Y_seq_len))
            self.training_logits = training_decoder_output.rnn_output
            
        with tf.variable_scope('decode', reuse=True):
            predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                embedding = decoder_embedding,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS)
            predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
                    cell = decoder_cells,
                    helper = predicting_helper,
                    initial_state = decoder_cells.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state),
                    output_layer = dense)
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder = predicting_decoder,
                    impute_finished = True,
                    maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
            self.greedy = predicting_decoder_output.sample_id
            self.greedy = tf.identity(self.greedy,name='greedy')
        
        with tf.variable_scope('decode', reuse=True):
            
            encoder_out_tiled = tf.contrib.seq2seq.tile_batch(encoder_out, beam_width)
            encoder_state_tiled = tf.contrib.seq2seq.tile_batch(encoder_state, beam_width)
            X_seq_len_tiled = tf.contrib.seq2seq.tile_batch(self.X_seq_len, beam_width)
            decoder_cell = attention(encoder_out_tiled, X_seq_len_tiled, reuse=True)
            
            predicting_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = decoder_cell,
                embedding = decoder_embedding,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS,
                initial_state = decoder_cell.zero_state(batch_size * beam_width, tf.float32).clone(
                    cell_state = encoder_state_tiled),
                beam_width = beam_width,
                output_layer = dense,
                length_penalty_weight = 0.0)
            
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = False,
                maximum_iterations = tf.reduce_max(self.X_seq_len))
            
            self.beam = predicting_decoder_output.predicted_ids[:, :, 0]
            self.beam = tf.identity(self.beam,name='beam')
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [7]:
size_layer = 256
num_layers = 2
embedded_size = 256
learning_rate = 1e-3
batch_size = 64
epoch = 20

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, 10000, 10000, learning_rate)
sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


In [9]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'lstm/model.ckpt')

'lstm/model.ckpt'

In [10]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [11]:
batch_x, _ = pad_sentence_batch(train_X[: 5], 0)
batch_y, _ = pad_sentence_batch(train_Y[: 5], 0)

In [12]:
sess.run([model.cost, model.optimizer], feed_dict = {model.X: batch_x, model.Y: batch_y})

[9.212443, None]

In [13]:
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = {model.X: test_X[:1]})

In [14]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x, seq_x = pad_sentence_batch(train_X[i : index], PAD)
        batch_y, seq_y = pad_sentence_batch(train_Y[i : index], PAD)
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
        
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x, seq_x = pad_sentence_batch(test_X[i : index], PAD)
        batch_y, seq_y = pad_sentence_batch(test_Y[i : index], PAD)
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 2/22905 [00:00<24:41, 15.46it/s, accuracy=0.994, cost=0.0118]

epoch 1, training avg loss 0.142025, training avg acc 0.967012
epoch 1, testing avg loss 0.046462, testing avg acc 0.985901


minibatch loop:   0%|          | 2/22905 [00:00<25:31, 14.96it/s, accuracy=0.994, cost=0.0408]

epoch 2, training avg loss 0.041392, training avg acc 0.987289
epoch 2, testing avg loss 0.037646, testing avg acc 0.988633


minibatch loop:   0%|          | 2/22905 [00:00<26:29, 14.41it/s, accuracy=1, cost=0.00266]

...

epoch 11, training avg loss 0.023402, training avg acc 0.993021
epoch 11, testing avg loss 0.029659, testing avg acc 0.991731


minibatch loop:   0%|          | 2/22905 [00:00<25:19, 15.08it/s, accuracy=1, cost=0.00244]

epoch 12, training avg loss 0.023212, training avg acc 0.993069
epoch 12, testing avg loss 0.031861, testing avg acc 0.991566


minibatch loop:  88%|████████▊ | 20204/22905 [23:32<03:03, 14.76it/s, accuracy=0.997, cost=0.00684]

KeyboardInterrupt: 

In [15]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'lstm-bahdanau/model.ckpt')

'lstm-bahdanau/model.ckpt'

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(training = False)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'transformer-tiny/model.ckpt')

In [16]:
import youtokentome as yttm
bpe = yttm.BPE(model='stemmer.model')

In [29]:
berjalankan = bpe.encode('berjalankansangat', output_type=yttm.OutputType.ID)

In [30]:
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = {model.X: [berjalankan]})

In [33]:
bpe.decode(predicted)

['jalan<EOS>']

In [34]:
bpe.decode(predicted_b)

['jalan<EOS>']

In [36]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name)
        and 'Adam' not in n.name
        and 'beta' not in n.name
        and 'OptimizeLoss' not in n.name
        and 'Global_Step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Variable',
 'Variable_1',
 'rnn/multi_rnn_cell/cell_0/lstm_cell/kernel',
 'rnn/multi_rnn_cell/cell_0/lstm_cell/bias',
 'rnn/multi_rnn_cell/cell_1/lstm_cell/kernel',
 'rnn/multi_rnn_cell/cell_1/lstm_cell/bias',
 'decode/memory_layer/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/bias',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_1/lstm_cell/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_1/lstm_cell/bias',
 'decode/decoder/attention_wrapper/bahdanau_attention/query_layer/kernel',
 'decode/decoder/attention_wrapper/bahdanau_attention/attention_v',
 'decode/decoder/attention_wrapper/attention_layer/kernel',
 'decode/decoder/dense/kernel',
 'decode/decoder/dense/bias',
 'decode_1/greedy',
 'decode_2/decoder/while/BeamSearchDecoderStep/beam_width',
 'decode_2/decoder/while/BeamSearchDecoderStep/next_beam_probs/range/start',
 

In [37]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"
    clear_devices = True
    with tf.Session(graph=tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(",")
        ) 
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

In [38]:
freeze_graph("lstm-bahdanau", strings)

Instructions for updating:
Use standard file APIs to check for files with this prefix.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from lstm-bahdanau/model.ckpt


INFO:tensorflow:Restoring parameters from lstm-bahdanau/model.ckpt


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


...

INFO:tensorflow:Froze 16 variables.


INFO:tensorflow:Froze 16 variables.


INFO:tensorflow:Converted 16 variables to const ops.


INFO:tensorflow:Converted 16 variables to const ops.


1649 ops in the final graph.


In [40]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [41]:
g=load_graph('lstm-bahdanau/frozen_model.pb')

In [43]:
x = g.get_tensor_by_name('import/Placeholder:0')
i_greedy = g.get_tensor_by_name('import/decode_1/greedy:0')
i_beam = g.get_tensor_by_name('import/decode_2/beam:0')

In [44]:
test_sess = tf.InteractiveSession(graph=g)

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [45]:
greedy, beam = test_sess.run([i_greedy, i_beam], feed_dict = {x: [berjalankan]})

In [50]:
bpe.decode(berjalankan)

['berjalankansangat']

In [49]:
bpe.decode(greedy.tolist()[0])

['jalan<EOS>']

In [51]:
bpe.decode(beam.tolist()[0])

['jalan<EOS><EOS><EOS><EOS><EOS>']

In [52]:
import boto3
s3 = boto3.client('s3')
bucketName = 'huseinhouse-storage'

Key = 'lstm-bahdanau/frozen_model.pb'
outPutname = "v34/stem/model.pb"
s3.upload_file(Key,bucketName,outPutname)

In [53]:
Key = 'stemmer.model'
outPutname = "v34/stem/bpe.model"
s3.upload_file(Key,bucketName,outPutname)